# *Startups operations/close predictions [M1_31DS+]*
_____

**Описание исследования.**

Практико-ориентированное соревнование по курсу Специалист по Data Science плюс. Исследование активности стартапов с целью построения модели классификации для решения задачи прогнозирования `удачности` стартапа.
_____
**Цель исследования.**
    
Провести анализ данных о стартапах за временной период с `1970-01-01` по `2018-01-01`. Выявить скрытые паттерны в данных, позволяющие более детально оценивать открытые на рынках стартапы. Разработать модель *бинарной классификации* для прогнозирования **успешности** стартапа.
_____
**Задачи исследования.**

1. Оценить уровень финансовой активности стартапов.
2. Выявить закономерности, влияющие на жизненный цикл стартапов, обнаружить скрытые зависимости.
3. Построить модели Машинного обучения (МО) -  (модель классификации) для предсказания **успешности** стартапов.

_____
**Исходные данные.**

Для исследования 2 файла со следующей информацией:

- `kaggle_startups_train_27042024.csv` - Основная таблица, содержащая данные о стартапах, содержит 53000 значений, представлена *14* признаками.
- `kaggle_startups_test_27042024.csv` - Таблица с тестовыми данными для оценки качества моделей.

**Признаковое описание данных**

# Список признаков:

- `name` - Название стартапа;
- `category_list` - Список категорий, к которым относится стартап;
- `funding_total_usd` - Общая сумма финансирования в USD;



______

**Содержание исследования.**

***Часть 1. Загрузка данных:***
* [1. Загрузка библиотек. Загрузка данных и их изучение. Анализ общей информации.](#pt1)
* [2. Предобработка данных.](#pt2)

<a id = 'pt1'></a>
# Загрузка данных

## Загрузка библиотек. Загрузка данных и их изучение. Анализ общей информации.

In [1]:
#Общие импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import tensorflow as tf
from utils import path_check, unn_info_pct
from nlp_funcs import clean_text, vectorize, semantic_similarity
from scipy import spatial

#Иморты sklearn
from sklearn.model_selection import (train_test_split, cross_val_score, GridSearchCV)
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

#Доп. библиотеки
#Корреляционный анализ
!pip install phik -q
from phik import phik_matrix
from phik.report import plot_correlation_matrix

#Анализ полученных моделей
!pip install shap -q
import shap

#Обработка естественного языка
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")

import gensim.downloader as api

#Служебные библиотеки
import os
import re
import string

import random
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore");

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gosha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gosha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''Обработка нежелательных элементов для задачи семантического анализа'''
stop_list = set(stopwords.words('english') + ['also', 'including', 'offering', 'equity'
    'tend', 'less', 'whose', 'involved', 'related', 'includes', 'engaged', 'using', 'contains'])

#### Данные о стартапах
Загружается основной датасет `kaggle_startups_train_27042024.csv` с обучающей выборкой, для которого будет выполнен исследовательский анализ данных.

In [3]:
raw_data = pd.read_csv(path_check('\\data\\kaggle_startups_train_27042024.csv'),
                   parse_dates=['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'])

OK - Path exists


In [4]:
print('Основная информация о датасете')
raw_data.info()
print()
print('Представление данных')
raw_data.head()

Основная информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52514 entries, 0 to 52513
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               52513 non-null  object        
 1   category_list      50049 non-null  object        
 2   funding_total_usd  42445 non-null  float64       
 3   status             52514 non-null  object        
 4   country_code       47013 non-null  object        
 5   state_code         45752 non-null  object        
 6   region             46156 non-null  object        
 7   city               46156 non-null  object        
 8   funding_rounds     52514 non-null  int64         
 9   founded_at         52514 non-null  datetime64[ns]
 10  first_funding_at   52514 non-null  datetime64[ns]
 11  last_funding_at    52514 non-null  datetime64[ns]
 12  closed_at          4915 non-null   datetime64[ns]
dtypes: datetime64[ns](4), float64(

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,828626.0,operating,CHE,25,Zurich,Zürich,2,2009-12-31,2011-05-01,2014-12-01,NaT
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.0,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaT
2,Reviva Pharmaceuticals,Biotechnology,35456381.0,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaT
3,Sancilio and Company,Health Care,22250000.0,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaT
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-09-30,2012-02-01,2012-02-01,NaT


Датасет содержит информацию о `52514` компаниях-стартапах, представлен `7 категориальными признаками`, `2 числовыми признаками`, `4 признаками типа datetime (даты)`. Рассматриваемый целевой признак `status` представлен категориальным признаком, который далее потребует кодирования.

В представленной информации о загруженных данных отражено наличие пропусков в данных, которые содержатся в признаках: `category_list`, `funding_total_usd`, `country_code`, `state_code`, `region`, `city`, `closed_at`. Для признака `closed_at` наличие пропусков не является критичным, так основная часть стартапов продолжала работу на момент выгрузки данных, поэтому пропуски в признаке будут заполнены датой выгрузки. Признаки `founded_at`, `closed_at` потенциально могут быть мало информативны для модели, при этом на основе данных признаков может быть сформирован новый признак: **длительность работы стартапа** - `lifetime`.

In [5]:
#Оценка количества пропусков
unn_info_pct(raw_data)

,0,1
name,1.0,0.001904
category_list,2465.0,4.693986
funding_total_usd,10069.0,19.173935
status,0.0,0.000000
country_code,5501.0,10.475302
state_code,6762.0,12.876566
region,6358.0,12.107248
city,6358.0,12.107248
funding_rounds,0.0,0.000000
founded_at,0.0,0.000000


Оценка количества пропусков показала, что в признаковом описании данных присутствует до 19.2% пропусков от общего числа данных. При этом наличие пропусков в признаке `closed_at` не является ошибкой, так как данный признак отражает факт того, что стартап продолжал работать на момент выгрузки данных.

#### Данные о стартапах - ТЕСТОВАЯ ВЫБОРКА
Загружается вторичный датасет `kaggle_startups_test_27042024.csv` с тестовой выборкой, который будет использоваться для валидации готовых моделей МО.

In [6]:
test_data_raw = pd.read_csv(path_check('\\data\\kaggle_startups_test_27042024.csv'),
                   parse_dates=['founded_at', 'first_funding_at', 'last_funding_at'])
print('Основная информация о датасете')
test_data_raw.info()
print()
print('Представление данных')
test_data_raw.head()

OK - Path exists
Основная информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               13125 non-null  object        
 1   category_list      12534 non-null  object        
 2   funding_total_usd  10547 non-null  float64       
 3   country_code       11743 non-null  object        
 4   state_code         11430 non-null  object        
 5   region             11536 non-null  object        
 6   city               11538 non-null  object        
 7   funding_rounds     13125 non-null  int64         
 8   founded_at         13125 non-null  datetime64[ns]
 9   first_funding_at   13125 non-null  datetime64[ns]
 10  last_funding_at    13125 non-null  datetime64[ns]
 11  lifetime           13125 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(6)
memory usage: 

,name,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,lifetime
0,Crystalsol,Clean Technology,2819200.0,NIC,17,NaN,NaN,1,2008-06-01,2009-07-01,2009-07-01,3501
1,JBI Fish & Wings,Hospitality,NaN,USA,TN,TN - Other,Humboldt,1,2010-07-25,2010-07-28,2010-07-28,2717
2,COINPLUS,Finance,428257.0,LUX,3,Esch-sur-alzette,Esch-sur-alzette,2,2014-06-16,2014-05-15,2014-09-18,1295
3,Imagine Communications,Software|Video|Video Streaming,34700000.0,USA,CA,San Diego,San Diego,4,2005-01-01,2005-01-01,2010-04-20,4748
4,DNA13,Software,4530000.0,CAN,ON,Ottawa,Ottawa,1,2001-01-01,2007-05-08,2007-05-08,6209


Датасет является содержит информацию о `13125` компаниях-стартапах, представлен `6 категориальными признаками`, `3 числовыми признаками`, `3 признаками типа datetime` будет использован для оценки качества разработанной модели МО. Представленные в датасете признаки отличаются от представленных в обучающем датасете.

В представленной информации о загруженных данных отражено наличие пропусков в данных, которые содержатся в признаках: `category_list`, `funding_total_usd`, `country_code`, `state_code`, `region`, `city`.

In [7]:
unn_info_pct(test_data_raw)

,0,1
name,0.0,0.000000
category_list,591.0,4.502857
funding_total_usd,2578.0,19.641905
country_code,1382.0,10.529524
state_code,1695.0,12.914286
region,1589.0,12.106667
city,1587.0,12.091429
funding_rounds,0.0,0.000000
founded_at,0.0,0.000000
first_funding_at,0.0,0.000000


Перед обработкой данных и анализом исходные датасеты копируются, работа по исследованию данных проводится над копиями датасетов.

In [8]:
data = raw_data.copy()
test_data = test_data_raw.copy()

<a id = 'pt2'></a>
# Предобработка данных

Полученные данные как для тестовой, так и для тренировочной выборок требуют тщательной предобработки и качественного анализа. В  данных наблюдается значительное число пропусков, наблюдается наличие смешанных данных для признака `category_list`, `region`.

### Датасет - ТРЕНИРОВОЧНАЯ ВЫБОРКА

Работа далее осуществляется с датасетом `data` - копией исходного тренировочного датасета.

#### Признак `name`
Отражает имя стартапа, является строковым признаком.

In [9]:
temp = data['name'].isna().sum()
print(f'Число пропусков признака name {temp}')
temp = data['name'].nunique()
print(f'Число уникальных значений признака name {temp}')

Число пропусков признака name 1
Число уникальных значений признака name 52513


Рассматриваемый признак `name` представлен полностью уникальными значениями, обнаружен *1* пропуск, который будет заполнен заглушкой - названием `Generic`.

In [10]:
data['name'] = data['name'].fillna('Generic')

#### Признак `category_list`

Обработка методами `NLP` происходит с помощью модели `Word2Vec` (используется [glove-wiki-gigaword-300](https://github.com/piskvorky/gensim-data?ref=dylancastillo.co#models)), расчёт косинусного расстояния методами `scipy.spatial.distance.cosine` для семантического анализа категорий, а также сравнения категорий с описаниями, выбранными из классификации `Global Industry Classification Sector (GICS)` (описания представлены по пути 'word2vec_sim/class_description.txt').

In [11]:
#загрузка Word2Vec
w2vec = api.load('glove-wiki-gigaword-300')

In [12]:
#Загрузка текстовых описаний категорий
with open(path_check('\\word2vec_sim\\class_description.txt'), 'r') as file: #импорт описаний из файла
    content = file.read().split('---\n')[1:]

cat_dict = {} #Словарь категорий
for i in range(len(content)): #Формирование описаний
    key = content[i].split('\n')[0].replace('Sector', '').strip()
    text = ' '.join(content[i].split('\n')[1:]).strip()
    token = clean_text(text, word_tokenize, stop_list)
    emb = vectorize(token, w2vec)
    cat_dict.update({key: pd.DataFrame([[i, text, token, emb]], columns = ['cat_num', 'desc', 'token', 'embedding'])})

OK - Path exists


In [13]:
data['category_list'] = data['category_list'].str.replace('|', ' ')
data['category_list'] = data['category_list'].fillna('')

In [14]:
data['tokens'] = data['category_list'].apply(lambda x: clean_text(x, word_tokenize, stop_list))
data['embedding'] = data['tokens'].apply(lambda x: vectorize(x, w2vec))
data['categories'] = data['embedding'].apply(lambda x: semantic_similarity(x, cat_dict)[0])

In [15]:
data['categories'].value_counts()

categories
Communication Services    17459
Information Technology    10678
Health Care                9580
Industrials                3872
Financials                 3028
Consumer Discretionary     2561
Generic                    2482
Real Estate                1197
Consumer Staples            821
Energy                      395
Utilities                   360
Materials                    81
Name: count, dtype: int64